In [ ]:
!pip install langchain langchain-community langchain-groq python-dotenv


In [1]:
pip install langchain openai datasets tiktoken qdrant-client pipwin langchain_community langchainhub groq pypdf 


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
import os
from dotenv import load_dotenv

from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import DocArrayInMemorySearch
from langchain_groq import ChatGroq  # Correct import for Groq now
from langchain.chains import RetrievalQA


In [ ]:
load_dotenv()
groq_api_key = os.getenv("GROQ_API_KEY")
if not groq_api_key:
    raise ValueError("Please set GROQ_API_KEY in environment or .env")

DOC_PATH = "source.md"
if not os.path.exists(DOC_PATH):
    with open(DOC_PATH, "w") as f:
        f.write("Couldn't find the source.md file. Please ensure it exists in the current directory.")

documents = TextLoader(DOC_PATH).load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
texts = text_splitter.split_documents(documents)

embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
vector_store = DocArrayInMemorySearch.from_documents(texts, embeddings)

llm = ChatGroq(
    model="llama3-8b-8192",  
    api_key=groq_api_key
)

qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=vector_store.as_retriever(),
    verbose=True
)

query = "Who works in QAZLaw?"
response = qa_chain.invoke(query)

print("\n Question:", query)
print(" Answer:", response['result'])




> Entering new RetrievalQA chain...

> Finished chain.

 Question: Who works in QAZLaw?
 Answer: According to the provided information, Nazym Abisheva, who is 30 years old and holds the role of Legal Counsel, works at QAZLaw.


In [1]:

!pip install cryptography 


[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
import os

from dotenv import load_dotenv
from cryptography.fernet import Fernet

In [2]:

from cryptography.fernet import Fernet

INPUT_FILE = "source_example1.md"
OUTPUT_FILE = INPUT_FILE + ".enc"

def encrypt_file(input_path, key, output_path):
    fernet = Fernet(key)
    with open(input_path, "rb") as f:
        plaintext = f.read()
    encrypted = fernet.encrypt(plaintext)
    with open(output_path, "wb") as f:
        f.write(encrypted)
    print(f"✅ Encrypted: {input_path} ➜ {output_path}")

def main():
    key = Fernet.generate_key()
    print("Encryption key (save this in .env as SECURE_KEY):")
    print(key.decode())

    encrypt_file(INPUT_FILE, key, OUTPUT_FILE)

if __name__ == "__main__":
    main()


🔐 Your encryption key (save this in .env as SECURE_KEY):
BoMqlCHadDyBeHas3GeSnUBQPHwVcvKHQVlLnNGGVCk=
✅ Encrypted: source_example1.md ➜ source_example1.md.enc


In [3]:
# secure_rag.py

import os
import sys
from dotenv import load_dotenv
from cryptography.fernet import Fernet, InvalidToken

from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import DocArrayInMemorySearch
from langchain_groq import ChatGroq
from langchain.chains import RetrievalQA

In [9]:

load_dotenv()
GROQ_API_KEY = os.getenv("GROQ_API_KEY")
SECURE_KEY = os.getenv("SECURE_KEY")

if not GROQ_API_KEY or not SECURE_KEY:
    sys.exit("Missing GROQ_API_KEY or SECURE_KEY in your .env file")


def decrypt_file(file_path, key):
    try:
        fernet = Fernet(key)
        with open(file_path, "rb") as f:
            encrypted = f.read()
        decrypted = fernet.decrypt(encrypted)
        return decrypted.decode("utf-8")
    except InvalidToken:
        sys.exit("Invalid decryption key! Double-check your SECURE_KEY.")
    except Exception as e:
        sys.exit(f"Failed to decrypt file: {e}")


print("Decrypting file...")
decrypted_text = decrypt_file("source_example1.md.enc", SECURE_KEY)

TEMP_FILE = "temp_decrypted.md"
with open(TEMP_FILE, "w", encoding="utf-8") as f:
    f.write(decrypted_text)

try:
    
    print("Loading and splitting document...")
    documents = TextLoader(TEMP_FILE).load()
    splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
    texts = splitter.split_documents(documents)

    
    print("Generating embeddings...")
    embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
    vector_store = DocArrayInMemorySearch.from_documents(texts, embeddings)

    print("Connecting to Groq (LLaMA 3 8B)...")
    llm = ChatGroq(model="llama3-8b-8192", api_key=GROQ_API_KEY)


    
    qa_chain = RetrievalQA.from_chain_type(
        llm=llm,
        retriever=vector_store.as_retriever(),
        verbose=False
    )



    query = "What is the most popular company?"
    print(f" Asking LLM: {query}")
    response = qa_chain.invoke(query)

    print("\nLLM Response:\n" + response["result"])





finally:
    if os.path.exists(TEMP_FILE):
        os.remove(TEMP_FILE)
        print("Cleaned up temporary decrypted file.")


Decrypting file...
Loading and splitting document...
Generating embeddings...
Connecting to Groq (LLaMA 3 8B)...
 Asking LLM: What is the most popular company?

LLM Response:
Based on the given information, QazaqDrinks is the most popular company, as it is mentioned as the company for most of the individuals, including Galymzhan Serikov (CEO & Founder), Zhansaya Muratova (Sales Representative), Bauyrzhan Kalykov (COO), Kairat Bekturov (Distribution Manager), Lazzat Ibraimova (Brand Ambassador), and Marat Ongarov (Packaging Designer).
Cleaned up temporary decrypted file.


ERROR: Could not find a version that satisfies the requirement sys (from versions: none)

[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip
ERROR: No matching distribution found for sys
